In [ ]:
import pandas as pd
import os
import re
import concurrent.futures
from openai import OpenAI
from tqdm import tqdm  # 进度条库，如果没安装请 pip install tqdm

# ================= 配置区域 =================

# 1. 大模型配置
LLM_API_KEY = "EMPTY"
LLM_API_BASE = "http://192.168.6.117:19534/v1"
MODEL_NAME = "Qwen/Qwen3-32B"

# 2. 并发数量 (根据你服务器的承受能力调整，建议 5-10)
MAX_WORKERS = 8 

# 3. 文件路径
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()

# 输入文件 (你刚刚上传的文件名)
INPUT_FILENAME = "api_retrieval_split_result.xlsx"
INPUT_PATH = os.path.join(CURRENT_DIR, INPUT_FILENAME)

# 输出文件
OUTPUT_PATH = os.path.join(CURRENT_DIR, "final_analysis_comparison.xlsx")

# ===========================================

# 初始化客户端 (在主进程初始化，但在线程中调用是安全的)
client = OpenAI(api_key=LLM_API_KEY, base_url=LLM_API_BASE)

def clean_model_output(text):
    """清洗模型输出，去除思考标签"""
    if not text: return ""
    cleaned_text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return cleaned_text.strip()

def merge_columns(row, prefix, count):
    """
    合并多个召回结果列为一个字符串
    例如: 召回结果1 + 召回结果2 ...
    """
    content_list = []
    for i in range(1, count + 1):
        # 兼容两种列名格式: "方案1_召回结果1" 或 "召回结果1"
        col_name = f"{prefix}{i}"
        
        # 如果找不到带前缀的，尝试找不带前缀的 (针对原问题召回结果)
        if col_name not in row and prefix == "召回结果":
             col_name = f"召回结果{i}"

        if col_name in row and pd.notna(row[col_name]):
            val = str(row[col_name]).strip()
            if val:
                content_list.append(f"[结果{i}]: {val}")
    
    return "\n".join(content_list)[:2000] # 截取前2000字防止Token溢出

def call_llm(prompt):
    """通用LLM调用函数"""
    try:
        chat_response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512
        )
        return clean_model_output(chat_response.choices[0].message.content)
    except Exception as e:
        return f"分析失败: {str(e)}"

def analyze_single_row(row):
    """
    处理单行数据的核心逻辑：执行4次分析
    """
    # 1. 获取基础数据
    question = row.get('问题', row.get('原问题', ''))
    
    # 获取三组召回内容
    # A. 原有召回 (召回结果1-5)
    baseline_content = merge_columns(row, "召回结果", 5)
    # B. 方案1召回 (方案1_召回结果1-3)
    scheme1_content = merge_columns(row, "方案1_召回结果", 3)
    # C. 方案2召回 (方案2_召回结果1-3)
    scheme2_content = merge_columns(row, "方案2_召回结果", 3)

    # --- 任务 1: 方案1 效果评估 ---
    prompt_eval_1 = f"""
请评估以下检索结果对问题的回答价值。
【问题】：{question}
【检索结果】：
{scheme1_content}

请简要评价：这些内容是否解决了问题？有哪些关键信息？(100字以内)
"""
    
    # --- 任务 2: 方案2 效果评估 ---
    prompt_eval_2 = f"""
请评估以下检索结果对问题的回答价值。
【问题】：{question}
【检索结果】：
{scheme2_content}

请简要评价：这些内容是否解决了问题？有哪些关键信息？(100字以内)
"""

    # --- 任务 3: 方案1 vs 原召回 对比 ---
    prompt_comp_1 = f"""
请对比两组检索结果的质量。
【问题】：{question}
【原召回结果】：
{baseline_content}
【方案1召回结果(单句扩写)】：
{scheme1_content}

请分析：方案1相比原召回结果，是提升了还是下降了？为什么？(100字以内，侧重准确性和相关性)
"""

    # --- 任务 4: 方案2 vs 原召回 对比 ---
    prompt_comp_2 = f"""
请对比两组检索结果的质量。
【问题】：{question}
【原召回结果】：
{baseline_content}
【方案2召回结果(潜在追问)】：
{scheme2_content}

请分析：方案2相比原召回结果，提供了哪些不同的视角？是更全面还是更杂乱？(100字以内)
"""

    # 串行调用4次 (单个线程内)
    # 也可以在这里再开并发，但控制整体行并发更简单
    res_eval_1 = call_llm(prompt_eval_1)
    res_eval_2 = call_llm(prompt_eval_2)
    res_comp_1 = call_llm(prompt_comp_1)
    res_comp_2 = call_llm(prompt_comp_2)

    return res_eval_1, res_eval_2, res_comp_1, res_comp_2

def main():
    print(f"正在读取文件: {INPUT_PATH}")
    if not os.path.exists(INPUT_PATH):
        print("❌ 文件不存在！")
        return

    df = pd.read_excel(INPUT_PATH)
    total_rows = len(df)
    print(f"共 {total_rows} 条数据，使用 {MAX_WORKERS} 线程并发处理...")

    # 准备结果容器
    results_map = {
        "方案1_效果评估": [],
        "方案2_效果评估": [],
        "方案1_vs_原召回_对比": [],
        "方案2_vs_原召回_对比": []
    }

    # 使用线程池并发处理
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # 将 DataFrame 转为字典列表方便处理
        rows = df.to_dict('records')
        
        # 提交任务
        # tqdm 用于显示进度条
        futures = list(tqdm(executor.map(analyze_single_row, rows), total=total_rows, desc="AI分析中"))

        # 收集结果
        for res in futures:
            results_map["方案1_效果评估"].append(res[0])
            results_map["方案2_效果评估"].append(res[1])
            results_map["方案1_vs_原召回_对比"].append(res[2])
            results_map["方案2_vs_原召回_对比"].append(res[3])

    # 将结果写入 DataFrame
    print("分析完成，正在写入 Excel...")
    for col, data in results_map.items():
        df[col] = data

    try:
        df.to_excel(OUTPUT_PATH, index=False)
        print(f"🎉 处理成功！结果保存在: {OUTPUT_PATH}")
    except PermissionError:
        print("❌ 保存失败：请关闭 Excel 文件后重试。")

if __name__ == "__main__":
    main()

正在读取文件: d:\jupyter\api_retrieval_split_result.xlsx
共 83 条数据，使用 8 线程并发处理...


AI分析中: 100%|██████████| 83/83 [53:04<00:00, 38.36s/it]  

分析完成，正在写入 Excel...
🎉 处理成功！结果保存在: d:\jupyter\final_analysis_comparison.xlsx
